<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [61]:
import pandas as pd
import psycopg2

In [63]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Предварительный анализ данных

1. Напишем запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [64]:


query_3_1 = f''' SELECT COUNT(*)
                FROM VACANCIES;
'''



In [65]:
df = pd.read_sql_query(query_3_1, connection)
df

C:\Users\bubul\AppData\Local\Temp\ipykernel_26220\3851344990.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_3_1, connection)


,count
0,49197


2. Напишем запрос, который посчитает количество работодателей (таблица employers). 

In [66]:
employers = f''' SELECT COUNT(*)
                FROM Employers;
'''

In [67]:

df = pd.read_sql_query(employers, connection)
df

C:\Users\bubul\AppData\Local\Temp\ipykernel_26220\4187801553.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(employers, connection)


,count
0,23501


3. Посчитаем с помощью запроса количество регионов (таблица areas).

In [68]:
regions =  f''' SELECT COUNT(*)
                FROM areas;
'''

In [69]:

df = pd.read_sql_query(regions, connection)
df

C:\Users\bubul\AppData\Local\Temp\ipykernel_26220\645395278.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(regions, connection)


,count
0,1362


4. Посчитаем с помощью запроса количество сфер деятельности в базе (таблица industries).

In [70]:

industries =  f''' SELECT COUNT(*)
                FROM industries;
'''

In [71]:
df = pd.read_sql_query(industries, connection)
df

C:\Users\bubul\AppData\Local\Temp\ipykernel_26220\1772076947.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(industries, connection)


,count
0,294


***

Проанализировав данные, нам прежде всего бросается в глаза количество вакансий. Они в 2 раза превышают количество работодателей.

#  Детальный анализ вакансий

1. Напишем запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируем по количеству вакансий в порядке убывания.

In [72]:


cnt_ = f'''SELECT 
                a.name area, 
                COUNT(*) cnt
            FROM areas a
            JOIN VACANCIES v on a.id = v.area_id 
            GROUP BY 
                a.name
            ORDER BY 
                cnt DESC
            LIMIT 
                5;
'''

In [73]:
df = pd.read_sql_query(cnt_, connection)
df

C:\Users\bubul\AppData\Local\Temp\ipykernel_26220\1060020076.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(cnt_, connection)


,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [74]:
salary = f'''SELECT COUNT(*) AS cnt
            FROM VACANCIES
            WHERE salary_from >=1 or salary_to >=1;
'''

In [75]:
df = pd.read_sql_query(salary, connection)
df

C:\Users\bubul\AppData\Local\Temp\ipykernel_26220\3858748318.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(salary, connection)


,cnt
0,24073


3. Найдем средние значения для нижней и верхней границы зарплатной вилки. Округлим значения до целого.

In [76]:
AVG_ratio = f'''SELECT 
                ROUND(AVG(salary_from)) low_salary,
                ROUND(AVG(salary_to)) high_salary
                FROM VACANCIES;
'''

In [77]:
df = pd.read_sql_query(AVG_ratio, connection)
df

C:\Users\bubul\AppData\Local\Temp\ipykernel_26220\262951354.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(AVG_ratio, connection)


,low_salary,high_salary
0,71065.0,110537.0


4. Напишем запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируем по убыванию количества.


In [78]:

schedule = f'''SELECT 
            COUNT(*) cnt,
            schedule,
            employment
            FROM VACANCIES
            GROUP BY schedule, employment 
            ORDER BY cnt DESC
            LIMIT 5;

                
'''


In [79]:
df = pd.read_sql_query(schedule, connection)
df

C:\Users\bubul\AppData\Local\Temp\ipykernel_26220\1725555365.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(schedule, connection)


,cnt,schedule,employment
0,35367,Полный день,Полная занятость
1,7802,Удаленная работа,Полная занятость
2,1593,Гибкий график,Полная занятость
3,1312,Удаленная работа,Частичная занятость
4,940,Сменный график,Полная занятость


5. Напишем запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [80]:
requre_experience = f''' SELECT 
                            COUNT(*) cnt,
                            experience "requre experience"
                        FROM VACANCIES
                        GROUP BY experience
                        ORDER BY cnt ASC;
                            

'''

In [81]:
df = pd.read_sql_query(requre_experience, connection)
df

C:\Users\bubul\AppData\Local\Temp\ipykernel_26220\1999453036.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(requre_experience, connection)


,cnt,requre experience
0,1337,Более 6 лет
1,7197,Нет опыта
2,14511,От 3 до 6 лет
3,26152,От 1 года до 3 лет


***

Мы можем сделать вывод о том, что Самые популярные города по количеству вакансий это Москва и Санкт-Петербург, что очевидно в принципе, средняя вилка зп находится от 71к рублей - 110к рублей. Также мы узнали что наибольшей популярностью пользуются вакансии на полный рабочий день и с полной занятостью, удалёнка у нас на 2 месте. Количество вакансий резко уменьшается если у нас большой опыт, чем он выше, тем вакансий меньше.

#  Анализ работодателей

1. Напишем запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [82]:
employers = f''' SELECT 
               COUNT(*) total,
               e.name
               FROM VACANCIES v 
               Join EMPLOYERS e on v.employer_id = e.id
               GROUP BY e.name
               ORDER BY total DESC
               
                            

'''


In [83]:
df = pd.read_sql_query(employers, connection)
df

C:\Users\bubul\AppData\Local\Temp\ipykernel_26220\3916006777.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(employers, connection)


,total,name
0,1933,Яндекс
1,491,Ростелеком
2,444,Тинькофф
3,428,СБЕР
4,331,Газпром нефть
...,...,...
14761,1,им. Тьюринга
14762,1,LLC Novalab Tech
14763,1,Компания Мир упаковки
14764,1,Лифинцев Олег Иванович


2. Напишем запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдём тот, в котором наибольшее количество работодателей.


In [84]:

employers_count = f'''SELECT 
                    COUNT(DISTINCT e.id) employer_count,
                    COUNT(DISTINCT v.id) vacancies_count,
                    a.name regions
                    FROM areas a
                    LEFT JOIN employers e on a.id = e.area
                    LEFT JOIN vacancies v on v.area_id = a.id
                    WHERE v.id is null
                    GROUP BY a.name, v.id
                    ORDER BY employer_count DESC
                    LIMIT 1;



'''

In [85]:
df = pd.read_sql_query(employers_count, connection)
df

C:\Users\bubul\AppData\Local\Temp\ipykernel_26220\453033041.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(employers_count, connection)


,employer_count,vacancies_count,regions
0,410,0,Россия


3. Для каждого работодателя посчитаем количество регионов, в которых он публикует свои вакансии. Отсортируем результат по убыванию количества.


In [86]:
areas_count = f'''SELECT 
                    COUNT(DISTINCT area_id) area_count,
                    e.name vacancies_name
                FROM VACANCIES v 
                JOIN employers e on v.employer_id = e.id
                GROUP BY e.name 
                ORDER BY area_count DESC
'''





In [87]:
df = pd.read_sql_query(areas_count, connection)
df

C:\Users\bubul\AppData\Local\Temp\ipykernel_26220\2511571736.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(areas_count, connection)


,area_count,vacancies_name
0,181,Яндекс
1,152,Ростелеком
2,116,Спецремонт
3,88,Поляков Денис Иванович
4,71,ООО ЕФИН
...,...,...
14761,1,UniSol
14762,1,UNISTORY LLC
14763,1,UNIT6
14764,1,United Distribution


4. Напишем запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [88]:

employer_id = f'''SELECT COUNT(*) AS count_employers_missing_industry 
                  FROM employers e 
                  LEFT JOIN employers_industries ei ON e.id = ei.employer_id 
                  WHERE e.area IS NOT NULL AND ei.employer_id IS NULL;
'''

In [89]:
df = pd.read_sql_query(employer_id, connection)
df

C:\Users\bubul\AppData\Local\Temp\ipykernel_26220\534405332.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(employer_id, connection)


,count_employers_missing_industry
0,8419


5. Напишем запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [90]:
company = f'''SELECT e.name 
FROM employers e 
JOIN employers_industries ei ON e.id = ei.employer_id 
GROUP BY e.id 
HAVING COUNT(DISTINCT ei.industry_id) = 4 
ORDER BY e.name 
LIMIT 1 OFFSET 2;

'''

In [91]:
df = pd.read_sql_query(company, connection)
df

C:\Users\bubul\AppData\Local\Temp\ipykernel_26220\3405284842.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(company, connection)


,name
0,2ГИС


6. С помощью запроса выясним, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [92]:
os = f''' SELECT 
          COUNT(EMPLOYERS.id) as employers_count
          FROM EMPLOYERS   
          JOIN EMPLOYERS_INDUSTRIES   on  employers.id = EMPLOYERS_INDUSTRIES.employer_id
          JOIN industries  on EMPLOYERS_INDUSTRIES.industry_id = industries .id
          Where industries.name = 'Разработка программного обеспечения'  
          

'''

In [93]:
df = pd.read_sql_query(os, connection)
df

C:\Users\bubul\AppData\Local\Temp\ipykernel_26220\325802786.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(os, connection)


,employers_count
0,3553


7. Для компании «Яндекс» выведим список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавим строку Total с общим количеством вакансий компании. Результат отсортируем по возрастанию количества.

Список городов-милионников возьмём [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 



In [94]:
import urllib.parse as req
import pandas as pd
import requests

path_common = req.urlparse('https://ru.wikipedia.org/wiki/Города-миллионеры_России#Список_городов-миллионеров')
url = path_common.scheme + '://' + req.quote(path_common.netloc) + req.quote(path_common.path) + '#' + req.quote(path_common.fragment)

df = pd.read_html(url)
df = df[1]
df = tuple(df['Город']) # выводим список городов в виде кортежа 
df 




('Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Екатеринбург',
 'Казань',
 'Нижний Новгород',
 'Красноярск',
 'Челябинск',
 'Самара',
 'Уфа',
 'Ростов-на-Дону',
 'Омск',
 'Краснодар',
 'Воронеж',
 'Пермь',
 'Волгоград')

In [95]:
import pandas as pd # второй способ 
import requests
from bs4 import BeautifulSoup

url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России#Список_городов-миллионеров'

html = requests.get(url).content
soup = BeautifulSoup(html, 'html.parser')
table = soup.find_all('table')[1]  
df = pd.read_html(str(table))[0]  

df = tuple(df['Город'])  
print(df)

('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Красноярск', 'Челябинск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград')


In [96]:
cities = f''' SELECT 
                    COUNT(DISTINCT v.id) vacancies_count,
                    a.name 
                    
              FROM VACANCIES v
              JOIN employers e on v.employer_id = e.id
              JOIN areas a on v.area_id = a.id 
              WHERE e.name = 'Яндекс' and a.name in {df} 
              GROUP BY a.name
              UNION  
              SELECT COUNT(DISTINCT v.id) vacancies_count,
              'Total' 
              FROM VACANCIES v
              JOIN employers e ON v.employer_id = e.id
              JOIN areas a ON v.area_id = a.id 
              WHERE e.name = 'Яндекс' 
              AND a.name IN {df};  
            
          

'''


In [97]:
cities = pd.read_sql_query(cities, connection)
cities

C:\Users\bubul\AppData\Local\Temp\ipykernel_26220\403033534.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cities = pd.read_sql_query(cities, connection)


,vacancies_count,name
0,21,Омск
1,22,Челябинск
2,23,Красноярск
3,24,Волгоград
4,25,Казань
5,25,Пермь
6,25,Ростов-на-Дону
7,26,Самара
8,26,Уфа
9,30,Краснодар


***

Вполне очевидно что самым крупным работодаттелем по количеству вакансий в регионах оказался Яндекс, у этой компании очень большой охват разной аудитории, в отличие от других компаний которые ограничиваются одной нишей. У 8419 компаний сфера деятельности не указана, это может быть связано с каким-либо мошенничеством. Компания 2ГИС имеет 4 сферы деятельности. Разработкой програмного обеспечения занимаются 3553 работодателей. В конце мы вывели количество вакансий для каждого города-миллионника. Общее количество вакансий среди них 485 

#  Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.




In [98]:
vacancies = f''' SELECT 
                    COUNT(DISTINCT id),
                    name vacancies_name
                FROM VACANCIES  
                WHERE LOWER(name) like '%data%' or LOWER(name) like '%данн%'
                GROUP BY name, id
                
                
''' 


In [99]:
vacancies = pd.read_sql_query(vacancies, connection)
vacancies

C:\Users\bubul\AppData\Local\Temp\ipykernel_26220\2991201902.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vacancies = pd.read_sql_query(vacancies, connection)


,count,vacancies_name
0,1,Аналитик данных (Департамент закупок)
1,1,Ведущий разработчик баз данных Oracle Pl/SQL
2,1,Разработчик баз данных
3,1,Аналитик данных
4,1,Администратор платформы больших данных (Big Da...
...,...,...
1766,1,Разработчик баз данных
1767,1,Data Analyst
1768,1,Data Analyst
1769,1,Data Analyst


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** На следующих этапах мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [100]:
useful_vacancies = f'''select
    v.name
from vacancies v
where
    (v.name ~* 'data scien(ce|tist)|исследователь данных|ML|machine learning|машинн.*обучен.*' and v.name !~* 'HTML')
    and (v.name ~* 'junior' or v.experience = 'Нет опыта' or v.employment = 'Стажировка')'''

In [101]:
useful_vacancies = pd.read_sql_query(useful_vacancies, connection)

useful_vacancies

C:\Users\bubul\AppData\Local\Temp\ipykernel_26220\1629809944.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  useful_vacancies = pd.read_sql_query(useful_vacancies, connection)


,name
0,Junior+ CV/ML developer
1,Data Scientist (Junior+)
2,Junior Data Scientist (Стажер)
3,"Математик, Junior/Middle/Senior Data Scientist"
4,Junior Data Scientist
5,Data Scientist/Machine Learning Engineer (Deve...
6,Data Scientist (Junior)
7,Intern/Junior Data Scientist
8,Junior Data Scientist (VK)
9,Junior/Middle Data Scientist


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

 Критерии для отнесения вакансии к DS указаны в предыдущем задании.

In [102]:
ds_vacancies=f''' select
        count(*)
    from
        vacancies
    where 
        (name ilike '%data scientist%' 
        or name ilike '%data science%'
        or name ilike '%исследователь данных%'
        or (name like '%ML%' and not name ilike '%HTML%')
        or name ilike '%machine learning%'
        or name ilike '%машинн%обучен%')
        and (key_skills ilike '%SQL%' or key_skills ilike '%postgres%')
 
'''
     
     
      

In [103]:
ds_vacancies = pd.read_sql_query(ds_vacancies, connection)
ds_vacancies

C:\Users\bubul\AppData\Local\Temp\ipykernel_26220\1853984032.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ds_vacancies = pd.read_sql_query(ds_vacancies, connection)


,count
0,201


4. Проверим, насколько популярен Python в требованиях работодателей к DS.Для этого вычислим количество вакансий, в которых в качестве ключевого навыка указан Python.



In [104]:
popular_python =f'''SELECT
        count(*) python_vacancies
    from
        vacancies 
    where 
        (name ilike '%data scientist%' 
        or name ilike '%data science%'
        or name ilike '%исследователь данных%'
        or (name like '%ML%' and not name ilike '%HTML%')
        or name ilike '%machine learning%'
        or name ilike '%машинн%обучен%')
        and (key_skills ilike '%Python%')
 
'''

In [105]:
popular_python = pd.read_sql_query(popular_python, connection)
popular_python

C:\Users\bubul\AppData\Local\Temp\ipykernel_26220\2827073350.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  popular_python = pd.read_sql_query(popular_python, connection)


,python_vacancies
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлим до двух знаков после точки-разделителя.

In [106]:
key_skills =f''' 
                SELECT 
                     round(avg(length(key_skills) - length(replace(key_skills, CHR(9), ''))+1), 2) cnt
                FROM vacancies v

where (name ilike '%data scientist%'

or name ilike '%data science%'
or name ilike '%исследователь данных%'
or (name like '%ML%' and name not ilike '%HTML%')
or name ilike '%machine learning%' or name ilike '%машинн%обучен%'
)
'''

In [107]:
key_skills = pd.read_sql_query(key_skills, connection)
key_skills

C:\Users\bubul\AppData\Local\Temp\ipykernel_26220\2500851053.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  key_skills = pd.read_sql_query(key_skills, connection)


,cnt
0,6.41


6. Напишем запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 


Выясним, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлим до целого числа. 

In [108]:
salary = f'''SELECT experience, ROUND(AVG(COALESCE((salary_from + salary_to) / 2, salary_from, salary_to)), 0) AS avg_salary
FROM vacancies 
where (name ilike '%data scientist%'
or name ilike '%data science%'
or name ilike '%исследователь данных%'
or (name like '%ML%' and name not ilike '%HTML%')
or name ilike '%machine learning%' or name ilike '%машинн%обучен%'
)
AND experience = 'От 3 до 6 лет' AND (salary_from IS NOT NULL OR salary_to IS NOT NULL)
GROUP BY experience
'''

In [109]:
salary = pd.read_sql_query(salary, connection)
salary

C:\Users\bubul\AppData\Local\Temp\ipykernel_26220\751990532.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  salary = pd.read_sql_query(salary, connection)


,experience,avg_salary
0,От 3 до 6 лет,243115.0


***

Исходя из наших вычислений мы можем сделать вывод. К данным имеет отношение 1771 вакансия. Для начинающего дата-сайентиста - 51. Это немного. А вот уже если мы включим в  качестве ключевого навыка  SQL или postgres, то вакансий станет почти в 4 раза больше- 201, правда стоит учесть тот факт, что мы убрали junior и поиск может не ограничиваться только начинающими специалистами. Количество вакансий в которых указан язык python - 351. Ключевых навыков для вакансий DS указывают в среднем 6.41. Мы выяснили на какую зп может претендовать кандидат с опытом работы от 3 до 6 лет -  243115, однако это уже не начинающий специалист. 

# Общий вывод по проекту

In [110]:
popular_vacancies = '''SELECT key_skills, COUNT(*) AS skill_count 
FROM vacancies 
GROUP BY key_skills 
ORDER BY skill_count DESC;'''

In [111]:
popular_vacancies = pd.read_sql_query(popular_vacancies, connection)
popular_vacancies

C:\Users\bubul\AppData\Local\Temp\ipykernel_26220\3760146379.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  popular_vacancies = pd.read_sql_query(popular_vacancies, connection)


,key_skills,skill_count
0,None,8456
1,1С программирование,94
2,Пользователь ПК\tРабота в команде\tРемонт ноут...,83
3,PostgreSQL\tSQL\tLinux\t1С программирование\tM...,71
4,Ответственность\tКлиентоориентированность,48
...,...,...
31214,Architecture\tArchimate\tСервис\tАрхитектура м...,1
31215,Arduino\tПайка\tЭлектроника\tМикроконтроллер,1
31216,ARIS\tBPMN\tIDEF\tРазработка технических задан...,1
31217,ARIS\tBPMN\tUML\tМоделирование бизнес процессо...,1


Тут мы можем увидеть, что наиболее популярным навыком(мы не берём в расчёт тех у кого они не указаны) является 1C программирование, потому что оно требуется везде, даже на работах не связанных с IT

In [112]:
demand_workers = f'''SELECT industries.name, COUNT(*) AS vacancy_count
FROM industries
JOIN employers_industries ON industries.id = employers_industries.industry_id
JOIN employers ON employers.id = employers_industries.employer_id
JOIN vacancies ON employers.id = vacancies.employer_id
GROUP BY industries.name
ORDER BY vacancy_count DESC;'''

In [113]:
demand_workers = pd.read_sql_query(demand_workers, connection)
demand_workers

C:\Users\bubul\AppData\Local\Temp\ipykernel_26220\1286624974.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  demand_workers = pd.read_sql_query(demand_workers, connection)


,name,vacancy_count
0,Разработка программного обеспечения,12499
1,"Системная интеграция, автоматизации технологи...",11034
2,"Интернет-компания (поисковики, платежные систе...",6413
3,Банк,2742
4,Мобильная связь,1585
...,...,...
288,Спортивная федерация,5
289,Химчистка,4
290,"Ассоциация в сфере культуры, искусства",2
291,Автошкола,1


In [114]:
connection.close()

Мы видим что у отрасли по Разработке ПО наибольший спрос на сотрудников

 Давайте сделаем общий вывод. Больше всего вакансий в Москве(5333) и Санкт-Петербурге(2851). У 24073 вакансий заполнено хотя бы 1 из 2 полей с зп. Средние значения нижней и верхней зарплатной вилки 71065 и 110537. Большинство вакансий у нас на полный рабочий день с полной занятостью. Количество вакансий уменьшается по мере роста опыта, вакансии в которых нет опыта не берём в счёт. Среди регионов, в которых нет вакансий, в котором наибольшее количество работодателей является Россия(410). Самой большой компанией является Яндекс, она публикует вакансии в 181 регионе. Количество работодателей у которых не указана сфера деятельности - 8419.  Среди компаний у которых указано 4 сферы деятельности, на 3 месте находится компания 2ГИС. У 3553 работодателей указана разработка ПО. Общее количество вакансий компании Яндекс для городов-миллионников 485, самым большим среди вакансий является Москва- 54. В наших данных есть 1771 которые имеют отношения к данным. Для начинающих дата-сайентистов доступно только 51 из них, что является довольно небольшим количеством. Однако, если использовать SQL или Postgres в качестве ключевого навыка, то количество вакансий почти увеличится в 4 раза до 201. Стоит учесть, что включение этих навыков может привести к тому, что поиск уже не будет ограничиваться только начинающими специалистами. Количество вакансий, где требуется использование языка Python, составляет 351. В среднем для вакансий, связанных с Data Science, требуется 6,41 ключевых навыков. Мы также определили, что кандидат с опытом работы от 3 до 6 лет может претендовать на заработную плату в размере 243115, но это уже не начинающий уровень.